#### This notebook demonstrates the use of InFoRM algorithms to mitigate bias for spectral clustering
InFoRM includes 3 algorithms, namely debiasing the input graph, debiasing the mining model and debiasing the mining result. We will show how to run all 3 algorithms for spectral clustering in this notebook.

## Get 2-view data

### Get vanilla clustering membership matrix first

In [1]:
# load necessary packages
import pickle5 as pickle
import load_graph
import utils
import random

import networkx as nx
import numpy as np

from scipy.sparse.csgraph import laplacian
from scipy.sparse.linalg import eigsh
import sklearn.preprocessing as skpp
from sklearn.cluster import k_means
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score

from sklearn.metrics import *
from scipy import sparse
from torch import optim
import torch

# load debias model
from method.debias_model_fair import DebiasModel
from data.synthetic.synthetic_data import *
from evaluate.sc import *

## Start with data

### synthetic

In [2]:
n_clusters = 2
adjs, sims, label  = synthetic_mat(1000)

adj_1 = sparse.coo_matrix(adjs[0]).asfptype()
adj_2 = sparse.coo_matrix(adjs[1]).asfptype()

sim_1 = sparse.coo_matrix(sims[0]).asfptype()
sim_2 = sparse.coo_matrix(sims[1]).asfptype()
graph_syn = {'adjs': [adj_1, adj_2], 
             'sims': [sim_1, sim_2]}

In [4]:
def vanilla(graph, name):
    udict = {}
    if name == 'graph':
        lcc = max(nx.connected_components(graph), key=len)  # take largest connected components
        adj = nx.to_scipy_sparse_matrix(graph, nodelist=lcc, dtype='float', format='csc')
    else:
        adj_1 = graph['adjs'][0]
        adj_2 = graph['adjs'][1]
    ## unnormalized laplacian
    lap_1 = laplacian(adj_1)
    lap_2 = laplacian(adj_2)
    
    lap1_minus = (-1) * lap_1
    lap2_minus = (-1) * lap_2
    #lap *= -1
    _, u_1 = eigsh(lap1_minus, which='LM', k=n_clusters, sigma=1.0)
    _, u_2 = eigsh(lap2_minus, which='LM', k=n_clusters, sigma=1.0)
    udict[name] = dict()
    u = [u_1, u_2]
    udict[name]['eigenvectors'] = u

    with open('result/sc/vanilla_'+name+'.pickle', 'wb') as f:
        pickle.dump(udict, f, protocol=pickle.HIGHEST_PROTOCOL)
    return u

In [21]:
vanilla_result = vanilla(syn, 'synthetic')
vanilla_kmeans = [KMeans(n_clusters=vanilla_result[0].shape[1], random_state=0, n_init=200).fit(vanilla_result[0]),
                 KMeans(n_clusters=vanilla_result[1].shape[1], random_state=0, n_init=200).fit(vanilla_result[1])]
vanilla_labels = [vanilla_kmeans[0].labels_, vanilla_kmeans[1].labels_]

In [6]:
## 2n+1 parameters
def spectral_clustering(adjs, sims, alpha_l, alpha_s, alpha_u, lambda_s, lambda_u, ncluster, niteration, v0,
                        max_patience, K_ini, K_min, mu,
                        risk_round_factor, reset_optimizer):
        ## initialize
        i = 0
        i_patience = 0

        alpha_l = np.array(alpha_l)
        alpha_s = np.array(alpha_s)
        alpha_u = np.array(alpha_u)

        L = {}
        U = {}
        
        ## save 
        matrices = {0: {}}
        matrices[0]['L'] = {}
        matrices[0]['U'] = {}
        matrices[0]['S'] = {}
        matrices[0]['risk'] = []
        matrices[0]['alpha_l'] = []
        matrices[0]['alpha_s'] = []
        matrices[0]['alpha_u'] = []
        matrices[0]['risk_best'] = []
        matrices[0]['alpha_s_best'] = []
        matrices[0]['alpha_u_best'] = []
        matrices[0]['params'] = []

        L_mod = np.zeros((adjs[0].shape[0], adjs[0].shape[1]))
        for v in range(len(adjs)):
            # initial U
            matrices[0]['S'][v] = alpha_s[v] * lambda_s * laplacian(sims[v]) 
            L[v] = alpha_l[v] * laplacian(adjs[v]) + alpha_s[v] * lambda_s * laplacian(sims[v])
            L_mod += L[v]
            L_minus = (-1) * L[v]

            _, U[v] = eigsh(L_minus, which='LM', k=ncluster, sigma=1.0, v0=v0)
        
        L_mod_minus = (-1) * L_mod
        _, U_mod = eigsh(L_mod_minus, which='LM', k=ncluster, sigma=1.0, v0=v0)
        
        K = K_ini
        print("start iter")
        while( i <= niteration) & (i_patience <= max_patience):
            matrices[i + 1] = {}
            matrices[i + 1]['L'] = {}
            matrices[i + 1]['U'] = {}
            matrices[i + 1]['S'] = {}
            matrices[i + 1]['KU'] = {}
            matrices[i + 1]['params'] = []
            matrices[i + 1]['risk'] = []
            matrices[i + 1]['alpha_l'] = []
            matrices[i + 1]['alpha_s'] = []
            matrices[i + 1]['alpha_u'] = []
            matrices[i + 1]['risk_best'] = []
            matrices[i + 1]['alpha_l_best'] = []
            matrices[i + 1]['alpha_s_best'] = []
            matrices[i + 1]['alpha_u_best'] = []
            #matrices[i + 1]['nmi_scores'] = {}
            #matrices[i + 1]['labels'] = {}
            
            alpha_l_penalty = (alpha_l).astype('float16')
            alpha_s_penalty = (alpha_s).astype('float16')
            alpha_u_penalty = (alpha_u).astype('float16')
            
            print('#### Iteration:', i, '; current alpha: ', alpha_l_penalty, alpha_s_penalty, alpha_u_penalty)
            loss = []
            
            
            for v in range(len(adjs)):
                u1 = sparse.csr_matrix(U[v])
                loss.append((u1.T @ L[v] @ u1).diagonal().sum())
            base_loss = np.array(loss)

            # h,risk,_ = bs_optimal(mu_i)
            risk = np.round(base_loss + 0, risk_round_factor)
            risk_max = np.max(risk)

            # argmax_risks
            argrisk_max = np.arange(risk.shape[0])
            argrisk_max = argrisk_max[risk == risk_max] #consider argmax exactly
        
            ## initialize
            if i == 0:
                risk_max_best = risk_max + 1
                
            if risk_max_best > risk_max: # risk is improved
                # update best risk
                risk_max_best = risk_max 
                argrisk_max_best = argrisk_max 
                risk_best = risk 
                alpha_l_best = alpha_l
                alpha_s_best = alpha_s
                alpha_u_best = alpha_u

                ## resets
                K = np.minimum(K, K_min)
                i_patience = 0
                type_step = 0 #improvement

                print('Iteration:', i,' k:',K,'Improved minimax risk (arg/max): ', argrisk_max_best, risk_max_best)
                #f.write('Iteration:', i,' k:',K,'Improved minimax risk (arg/max): ', argrisk_max_best, risk_max_best)
            else:
                K += 1
                i_patience += 1
                type_step = 1 # no improvement
                print('Iteration: ', i,' k:',K, 'No minimax risk improvement, current best (arg/max): ', 
                      argrisk_max_best, risk_max_best)
                #f.write('Iteration: ', i,' k:',K, 'No minimax risk improvement, current best (arg/max): ', 
                #      argrisk_max_best, risk_max_best)
            
            ## step update
            mask_alpha_l = np.zeros(alpha_l.shape)
            mask_alpha_s = np.zeros(alpha_s.shape)
            mask_alpha_u = np.zeros(alpha_u.shape)
            
            mask_alpha_l[risk >= risk_max_best] = 1
            mask_alpha_s[risk >= risk_max_best] = 1
            mask_alpha_u[risk >= risk_max_best] = 1
        
            step_alpha_l = mask_alpha_l / np.sum(mask_alpha_l)
            step_alpha_s = mask_alpha_s / np.sum(mask_alpha_s)
            step_alpha_u = mask_alpha_u / np.sum(mask_alpha_u)
            print('all_masks', mask_alpha_l, np.sum(mask_alpha_l), np.sum(mask_alpha_s), np.sum(mask_alpha_u))
                
            ##############    Save lists  ##############
            #weight vector updated after save lists
            matrices[i + 1]['params'].append([type_step, K, mu])
            matrices[i + 1]['risk'].append(risk)
            matrices[i + 1]['alpha_l'].append(alpha_l)
            matrices[i + 1]['alpha_s'].append(alpha_s)
            matrices[i + 1]['alpha_u'].append(alpha_u)
            matrices[i + 1]['risk_best'].append(risk_best)
            matrices[i + 1]['alpha_l_best'].append(alpha_l_best)
            matrices[i + 1]['alpha_s_best'].append(alpha_s_best)
            matrices[i + 1]['alpha_u_best'].append(alpha_u_best)
            
            print('Iteration:', i, '; step reduced minimax?:', type_step, ' risk (arg/ max)', argrisk_max,
                  risk[argrisk_max], '; (arg/ max best): ', argrisk_max_best, risk_max_best)
            print('alpha_l: ', alpha_l, 'alpha_s: ', alpha_s, '; new delta: ', step_alpha_l, step_alpha_s, ' mu: ', mu, '; K: ', K)
            print('risks: ', risk, ' ; best risk: ', risk_best)
            print()

            
            ############################################
            """for view, u in U.items():
                view_kmeans = KMeans(n_clusters=u.shape[1], random_state=42, n_init=5).fit(u)
                view_labels = view_kmeans.labels_
                matrices[i + 1]['labels'][view] = view_labels
                #matrices[i + 1]['nmi_scores'][view] = normalized_mutual_info_score(view_labels, vanilla_labels[view])
                matrices[i + 1]['nmi_scores'][view] = normalized_mutual_info_score(label, view_labels)
            """
            view_kmeans = KMeans(n_clusters=U_mod.shape[1], random_state=42, n_init=5).fit(U_mod)
            view_labels = view_kmeans.labels_
            matrices[i + 1]['labels'] = view_labels
            matrices[i + 1]['nmi_scores'] = normalized_mutual_info_score(label, view_labels)
            matrices[i + 1]['acc'] = acc(label, view_labels)
            
            print("NMI scores: ", matrices[i + 1]['nmi_scores'], "acc: ", matrices[i + 1]['acc'])

            ### UPDATE WEIGHTING_VECTOR ###
            alpha_l = (1 - mu) * alpha_l + step_alpha_l * mu / K
            alpha_s = (1 - mu) * alpha_s + step_alpha_s * mu / K
            alpha_u = (1 - mu) * alpha_u + step_alpha_u * mu / K
            

            i += 1
            L_mod = np.zeros((adjs[0].shape[0], adjs[0].shape[1]))
            L_s = np.zeros((adjs[0].shape[0], adjs[0].shape[1]))
            for j in range(len(adjs)):
                KU = sum(np.matmul(U[k], U[k].T) for k in range(len(U)) if k != j)
                lap = alpha_l[j] * L[j] - alpha_u[j] * lambda_u * KU + alpha_s[j] * lambda_s * laplacian(sims[j]) 
                L_mod += lap
                lap_minus = (-1) * lap
                _, U[j] = eigsh(lap_minus, which='LM', k=ncluster, sigma=1.0, v0=v0)
                L[j] = np.array(lap)

                matrices[i]['L'][j] = L[j]
                matrices[i]['U'][j] = U[j]
                matrices[i]['KU'][j] = KU
                matrices[i]['S'][j] = alpha_s[j] * lambda_s * laplacian(sims[j]) 
                L_s += matrices[i]['S'][j]
            L_mod_minus = (-1) * L_mod
            _, U_mod = eigsh(L_mod_minus, which='LM', k=ncluster, sigma=1.0, v0=v0)
            matrices[i]['L_mod'] = L_mod
            matrices[i]['U_mod'] = U_mod
            matrices[i]['L_s'] = L_s
            print('Iteration: ', i+1)


        #return matrices
        return matrices
    
def debias_mining_model(adjs, sims, name, alpha, lmbda, niter=20, metric=None):
    adjs = np.array(adjs)
    sims = np.array(sims)
    # debias spectral clustering
    
    alpha_l = [alpha, alpha]
    alpha_s = [alpha, alpha]
    alpha_u = [alpha, alpha]
    lambda_s = lmbda
    lambda_u = lmbda
    
    # V, U = sc.debias_alg(adj, sim, alpha, ncluster=10, v0=v0[name])
    # model = DebiasModelFair(adjs, sims, ncluster=10, niteration=niter, v0=v0[name])
    result = spectral_clustering(adjs, sims, alpha_l, alpha_s, alpha_u, lambda_s, lambda_u,
                                 ncluster=n_clusters, niteration=niter, v0=None,
                                 max_patience=15, K_ini=1, K_min=20, mu=0.5,
                                 risk_round_factor=3, reset_optimizer=False)
    

    print('dataset: {}\t metric: {} similarity'.format(name, metric))
    print('Finished!')

    return result

In [ ]:
## evaluation
def lp_diff(vanilla_result, fair_result, ord=None):
    diffs = {}
    if ord == 'fro':
        diffs = 0
        for i in range(vanilla_result.shape[1]):
            residual = min(
                np.linalg.norm(vanilla_result[:, i] + fair_result[:, i], ord=2),
                np.linalg.norm(vanilla_result[:, i] - fair_result[:, i], ord=2)
            )
            diffs += (residual ** 2)
        return np.sqrt(diffs)
    else:
        diff = vanilla_result - fair_result
        return np.linalg.norm(diff, ord=ord)
    
def calc_bias(ls, vanilla_result, fair_result):
    
    # calculate bias
    vanilla_bias = utils.trace(vanilla_result.T @ ls @ vanilla_result)  # vanilla bias
    fair_bias = utils.trace(fair_result.T @ ls @ fair_result)  # fair bias
    reduction = 1 - (fair_bias / vanilla_bias)
    return reduction

def acc(pred, true):
    acc = 0
    for i in range(len(pred)):
        if pred[i] == true[i]:
            acc += 1
    return acc / len(pred)

In [8]:
alpha = 0.1
lmbda = 0.5
result = dict()

result['syn'] = debias_mining_model(adjs, sims, name='syn', alpha=alpha, lmbda=lmbda, metric='jaccard')
with open('result/sc/model/jaccard_syn.pickle', 'wb') as f:
    pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)


start iter
#### Iteration: 0 ; current alpha:  [0.1 0.1] [0.1 0.1] [0.1 0.1]
Iteration: 0  k: 1 Improved minimax risk (arg/max):  [1] 81.762
all_masks [0. 1.] 1.0 1.0 1.0
Iteration: 0 ; step reduced minimax?: 0  risk (arg/ max) [1] [81.762] ; (arg/ max best):  [1] 81.762
alpha_l:  [0.1 0.1] alpha_s:  [0.1 0.1] ; new delta:  [0. 1.] [0. 1.]  mu:  0.5 ; K:  1
risks:  [32.392 81.762]  ; best risk:  [32.392 81.762]

NMI scores:  0.001381429316719824 acc:  0.503
Iteration:  2
#### Iteration: 1 ; current alpha:  [0.05 0.55] [0.05 0.55] [0.05 0.55]
Iteration:  1  k: 2 No minimax risk improvement, current best (arg/max):  [1] 81.762
all_masks [0. 1.] 1.0 1.0 1.0
Iteration: 1 ; step reduced minimax?: 1  risk (arg/ max) [1] [212.094] ; (arg/ max best):  [1] 81.762
alpha_l:  [0.05 0.55] alpha_s:  [0.05 0.55] ; new delta:  [0. 1.] [0. 1.]  mu:  0.5 ; K:  2
risks:  [ 11.305 212.094]  ; best risk:  [32.392 81.762]

NMI scores:  0.022137381541747343 acc:  0.537
Iteration:  3
#### Iteration: 2 ; curre

Iteration:  17
#### Iteration: 16 ; current alpha:  [1.55e-06 1.43e-01] [1.55e-06 1.43e-01] [1.55e-06 1.43e-01]
Iteration: 16  k: 7 Improved minimax risk (arg/max):  [1] 47.535
all_masks [0. 1.] 1.0 1.0 1.0
Iteration: 16 ; step reduced minimax?: 0  risk (arg/ max) [1] [47.535] ; (arg/ max best):  [1] 47.535
alpha_l:  [1.52587891e-06 1.42930748e-01] alpha_s:  [1.52587891e-06 1.42930748e-01] ; new delta:  [0. 1.] [0. 1.]  mu:  0.5 ; K:  7
risks:  [ 0.    47.535]  ; best risk:  [ 0.    47.535]

NMI scores:  0.18019488838176945 acc:  0.742
Iteration:  18
#### Iteration: 17 ; current alpha:  [7.75e-07 1.43e-01] [7.75e-07 1.43e-01] [7.75e-07 1.43e-01]
Iteration: 17  k: 7 Improved minimax risk (arg/max):  [1] 47.518
all_masks [0. 1.] 1.0 1.0 1.0
Iteration: 17 ; step reduced minimax?: 0  risk (arg/ max) [1] [47.518] ; (arg/ max best):  [1] 47.518
alpha_l:  [7.62939453e-07 1.42893946e-01] alpha_s:  [7.62939453e-07 1.42893946e-01] ; new delta:  [0. 1.] [0. 1.]  mu:  0.5 ; K:  7
risks:  [ 0.    4

### evaluate for synthetic

In [27]:
#vanilla_result = result['vanilla_ppi']
l = result['syn'][21]['L_mod']
ls = result['syn'][21]['L_s']
u = result['syn'][21]['U_mod']
fair_labels = result['syn'][21]['labels']

#vanilla_kmeans = KMeans(n_clusters=u.shape[1], random_state=42, n_init=5).fit(vanilla_result[0])
#vanilla_labels = vanilla_kmeans.labels_

# reduce
for k in range(2):
    bias = calc_bias(ls, vanilla_result[k], u)
    print('reduce_{}:'.format(k), bias)
    
# accuracy
print(acc(fair_labels, label), acc(vanilla_labels[0], label))

# f1-score
print(f1_score(label, fair_labels, average='macro'), f1_score(label, vanilla_labels[0], average='macro'))

# nmi
print(normalized_mutual_info_score(label, fair_labels), normalized_mutual_info_score(label, vanilla_labels[0]))

# ari
print(adjusted_rand_score(label, fair_labels), adjusted_rand_score(label, vanilla_labels[1]))

reduce_0: 0.06337793597021757
reduce_1: 0.06838971008411909
0.742 0.499
0.7411291584891575 0.33288859239492996
0.18019488838176945 0.001978869673592188
0.23349905614899846 0.0


## IMDB

In [6]:
def vanilla(graph, name, n_clusters):
    udict = {}
    if name == 'graph':
        lcc = max(nx.connected_components(graph), key=len)  # take largest connected components
        adj = nx.to_scipy_sparse_matrix(graph, nodelist=lcc, dtype='float', format='csc')
    elif name == 'synthetic':
        adj_1 = graph['adjs'][0]
        adj_2 = graph['adjs'][1]
        adjs = [adj_1, adj_2]
    elif name == 'imdb':
        #lcc_1 = max(nx.connected_components(graph[0]), key=len)  # take largest connected components
        #lcc_2 = max(nx.connected_components(graph[1]), key=len)
        adj_1 = nx.to_scipy_sparse_matrix(graph[0], nodelist=set(graph[0].nodes), dtype='float', format='csc')
        adj_2 = nx.to_scipy_sparse_matrix(graph[1], nodelist=set(graph[1].nodes), dtype='float', format='csc')
        adjs = [adj_1, adj_2]
    elif name == 'dblp':
        #lcc_1 = max(nx.connected_components(graph[0]), key=len)  # take largest connected components
        #lcc_2 = max(nx.connected_components(graph[1]), key=len)
        adj_1 = nx.to_scipy_sparse_matrix(graph[0], nodelist=set(graph[0].nodes), dtype='float', format='csc')
        adj_2 = nx.to_scipy_sparse_matrix(graph[1], nodelist=set(graph[1].nodes), dtype='float', format='csc')
        adj_3 = nx.to_scipy_sparse_matrix(graph[2], nodelist=set(graph[2].nodes), dtype='float', format='csc')
        adjs = [adj_1, adj_2, adj_3]
    ## unnormalized laplacian
    laps = [laplacian(adj) for adj in adjs]
    
    laps_minus = [(-1) * lap for lap in laps]
    #lap *= -1
    results = [eigsh(lap_minus, which='LM', k=n_clusters, sigma=1.0) for lap_minus in laps_minus]
    u = [results[i][1] for i in range(len(results))]
    udict[name] = dict()
    udict[name]['eigenvectors'] = u

    with open('result/sc/vanilla_'+name+'.pickle', 'wb') as f:
        pickle.dump(udict, f, protocol=pickle.HIGHEST_PROTOCOL)
    return u

## 2n+1 parameters
def spectral_clustering(adjs, sims, alpha_l, alpha_s, alpha_u, lambda_s, lambda_u, ncluster, niteration, v0,
                        max_patience, K_ini, K_min, mu,
                        risk_round_factor, reset_optimizer):
        ## initialize
        i = 0
        i_patience = 0

        alpha_l = np.array(alpha_l)
        alpha_s = np.array(alpha_s)
        alpha_u = np.array(alpha_u)

        L = {}
        U = {}
        
        ## save 
        matrices = {0: {}}
        matrices[0]['L'] = {}
        matrices[0]['U'] = {}
        matrices[0]['S'] = {}
        matrices[0]['risk'] = []
        matrices[0]['alpha_l'] = []
        matrices[0]['alpha_s'] = []
        matrices[0]['alpha_u'] = []
        matrices[0]['risk_best'] = []
        matrices[0]['alpha_s_best'] = []
        matrices[0]['alpha_u_best'] = []
        matrices[0]['params'] = []

        L_mod = np.zeros((adjs[0].shape[0], adjs[0].shape[1]))
        for v in range(len(adjs)):
            # initial U
            matrices[0]['S'][v] = alpha_s[v] * lambda_s * laplacian(sims[v]) 
            L[v] = alpha_l[v] * laplacian(adjs[v]) + alpha_s[v] * lambda_s * laplacian(sims[v])
            L_mod += L[v]
            L_minus = (-1) * L[v]

            _, U[v] = eigsh(L_minus, which='LM', k=ncluster, sigma=1.0, v0=v0)
        
        L_mod_minus = (-1) * L_mod
        _, U_mod = eigsh(L_mod_minus, which='LM', k=ncluster, sigma=1.0, v0=v0)
        
        K = K_ini
        print("start iter")
        while( i <= niteration) & (i_patience <= max_patience):
            matrices[i + 1] = {}
            matrices[i + 1]['L'] = {}
            matrices[i + 1]['U'] = {}
            matrices[i + 1]['S'] = {}
            matrices[i + 1]['KU'] = {}
            matrices[i + 1]['params'] = []
            matrices[i + 1]['risk'] = []
            matrices[i + 1]['alpha_l'] = []
            matrices[i + 1]['alpha_s'] = []
            matrices[i + 1]['alpha_u'] = []
            matrices[i + 1]['risk_best'] = []
            matrices[i + 1]['alpha_l_best'] = []
            matrices[i + 1]['alpha_s_best'] = []
            matrices[i + 1]['alpha_u_best'] = []
            #matrices[i + 1]['nmi_scores'] = {}
            #matrices[i + 1]['labels'] = {}
            
            alpha_l_penalty = (alpha_l).astype('float16')
            alpha_s_penalty = (alpha_s).astype('float16')
            alpha_u_penalty = (alpha_u).astype('float16')
            
            print('#### Iteration:', i, '; current alpha: ', alpha_l_penalty, alpha_s_penalty, alpha_u_penalty)
            loss = []
            
            
            for v in range(len(adjs)):
                u1 = sparse.csr_matrix(U[v])
                loss.append((u1.T @ L[v] @ u1).diagonal().sum())
            base_loss = np.array(loss)

            # h,risk,_ = bs_optimal(mu_i)
            risk = np.round(base_loss + 0, risk_round_factor)
            risk_max = np.max(risk)

            # argmax_risks
            argrisk_max = np.arange(risk.shape[0])
            argrisk_max = argrisk_max[risk == risk_max] #consider argmax exactly
        
            ## initialize
            if i == 0:
                risk_max_best = risk_max + 1
                
            if risk_max_best > risk_max: # risk is improved
                # update best risk
                risk_max_best = risk_max 
                argrisk_max_best = argrisk_max 
                risk_best = risk 
                alpha_l_best = alpha_l
                alpha_s_best = alpha_s
                alpha_u_best = alpha_u

                ## resets
                K = np.minimum(K, K_min)
                i_patience = 0
                type_step = 0 #improvement

                print('Iteration:', i,' k:',K,'Improved minimax risk (arg/max): ', argrisk_max_best, risk_max_best)
                #f.write('Iteration:', i,' k:',K,'Improved minimax risk (arg/max): ', argrisk_max_best, risk_max_best)
            else:
                K += 1
                i_patience += 1
                type_step = 1 # no improvement
                print('Iteration: ', i,' k:',K, 'No minimax risk improvement, current best (arg/max): ', 
                      argrisk_max_best, risk_max_best)
                #f.write('Iteration: ', i,' k:',K, 'No minimax risk improvement, current best (arg/max): ', 
                #      argrisk_max_best, risk_max_best)
            
            ## step update
            mask_alpha_l = np.zeros(alpha_l.shape)
            mask_alpha_s = np.zeros(alpha_s.shape)
            mask_alpha_u = np.zeros(alpha_u.shape)
            
            mask_alpha_l[risk >= risk_max_best] = 1
            mask_alpha_s[risk >= risk_max_best] = 1
            mask_alpha_u[risk >= risk_max_best] = 1
        
            step_alpha_l = mask_alpha_l / np.sum(mask_alpha_l)
            step_alpha_s = mask_alpha_s / np.sum(mask_alpha_s)
            step_alpha_u = mask_alpha_u / np.sum(mask_alpha_u)
            print('all_masks', mask_alpha_l, np.sum(mask_alpha_l), np.sum(mask_alpha_s), np.sum(mask_alpha_u))
                
            ##############    Save lists  ##############
            #weight vector updated after save lists
            matrices[i + 1]['params'].append([type_step, K, mu])
            matrices[i + 1]['risk'].append(risk)
            matrices[i + 1]['alpha_l'].append(alpha_l)
            matrices[i + 1]['alpha_s'].append(alpha_s)
            matrices[i + 1]['alpha_u'].append(alpha_u)
            matrices[i + 1]['risk_best'].append(risk_best)
            matrices[i + 1]['alpha_l_best'].append(alpha_l_best)
            matrices[i + 1]['alpha_s_best'].append(alpha_s_best)
            matrices[i + 1]['alpha_u_best'].append(alpha_u_best)
            
            print('Iteration:', i, '; step reduced minimax?:', type_step, ' risk (arg/ max)', argrisk_max,
                  risk[argrisk_max], '; (arg/ max best): ', argrisk_max_best, risk_max_best)
            print('alpha_l: ', alpha_l, 'alpha_s: ', alpha_s, '; new delta: ', step_alpha_l, step_alpha_s, ' mu: ', mu, '; K: ', K)
            print('risks: ', risk, ' ; best risk: ', risk_best)
            print()

            
            ############################################
            """for view, u in U.items():
                view_kmeans = KMeans(n_clusters=u.shape[1], random_state=42, n_init=5).fit(u)
                view_labels = view_kmeans.labels_
                matrices[i + 1]['labels'][view] = view_labels
                #matrices[i + 1]['nmi_scores'][view] = normalized_mutual_info_score(view_labels, vanilla_labels[view])
                matrices[i + 1]['nmi_scores'][view] = normalized_mutual_info_score(label, view_labels)
            """
            view_kmeans = KMeans(n_clusters=U_mod.shape[1], random_state=42, n_init=5).fit(U_mod)
            view_labels = view_kmeans.labels_
            matrices[i + 1]['labels'] = view_labels
            matrices[i + 1]['nmi_scores'] = normalized_mutual_info_score(label, view_labels)
            matrices[i + 1]['acc'] = acc(label, view_labels)
            
            print("NMI scores: ", matrices[i + 1]['nmi_scores'], "acc: ", matrices[i + 1]['acc'])

            ### UPDATE WEIGHTING_VECTOR ###
            alpha_l = (1 - mu) * alpha_l + step_alpha_l * mu / K
            alpha_s = (1 - mu) * alpha_s + step_alpha_s * mu / K
            alpha_u = (1 - mu) * alpha_u + step_alpha_u * mu / K
            

            i += 1
            L_mod = np.zeros((adjs[0].shape[0], adjs[0].shape[1]))
            L_s = np.zeros((adjs[0].shape[0], adjs[0].shape[1]))
            for j in range(len(adjs)):
                KU = sum(np.matmul(U[k], U[k].T) for k in range(len(U)) if k != j)
                lap = alpha_l[j] * L[j] - alpha_u[j] * lambda_u * KU + alpha_s[j] * lambda_s * laplacian(sims[j]) 
                L_mod += lap
                lap_minus = (-1) * lap
                _, U[j] = eigsh(lap_minus, which='LM', k=ncluster, sigma=1.0, v0=v0)
                L[j] = np.array(lap)

                matrices[i]['L'][j] = L[j]
                matrices[i]['U'][j] = U[j]
                matrices[i]['KU'][j] = KU
                matrices[i]['S'][j] = alpha_s[j] * lambda_s * laplacian(sims[j]) 
                L_s += matrices[i]['S'][j]
            L_mod_minus = (-1) * L_mod
            _, U_mod = eigsh(L_mod_minus, which='LM', k=ncluster, sigma=1.0, v0=v0)
            matrices[i]['L_mod'] = L_mod
            matrices[i]['U_mod'] = U_mod
            matrices[i]['L_s'] = L_s
            print('Iteration: ', i+1)



        #return matrices
        return matrices
    
def debias_mining_model(graphs, name, alpha_l, alpha_s, alpha_u, lambda_s, lambda_u, ncluster, niter=10, metric=None):
    udict = {}

    if name == 'synthetic':
        adj_1 = graphs['adjs'][0]
        adj_2 = graphs['adjs'][1]
        
        sim_1 = graphs['sims'][0]
        sim_2 = graphs['sims'][0]
        
        adjs = np.array([adj_1, adj_2])
        sims = np.array([sim_1, sim_2])
    elif name == 'imdb':
        # build adjacency matrix
        adj_1 = nx.to_scipy_sparse_matrix(graphs[0], nodelist=set(graphs[0].nodes), dtype='float', format='csc')
        adj_2 = nx.to_scipy_sparse_matrix(graphs[1], nodelist=set(graphs[1].nodes), dtype='float', format='csc')

        # build similarity matrix
        sim_1 = utils.get_similarity_matrix(adj_1, metric=metric)
        sim_2 = utils.get_similarity_matrix(adj_2, metric=metric)
    
        adjs = np.array([adj_1, adj_2])
        sims = np.array([sim_1, sim_2])
    
    # debias spectral clustering
    
    #alpha_l = alpha_
    #alpha_s = [alpha, alpha]
    #alpha_u = [alpha, alpha]
    #lambda_s = lmbda
    #lambda_u = lmbda
    
    # V, U = sc.debias_alg(adj, sim, alpha, ncluster=10, v0=v0[name])
    # model = DebiasModelFair(adjs, sims, ncluster=10, niteration=niter, v0=v0[name])
    result = spectral_clustering(adjs, sims, alpha_l, alpha_s, alpha_u, lambda_s, lambda_u,
                                 ncluster=n_clusters, niteration=niter, v0=None,
                                 max_patience=15, K_ini=1, K_min=20, mu=0.5,
                                 risk_round_factor=3, reset_optimizer=False)
    

    print('dataset: {}\t metric: {} similarity'.format(name, metric))
    print('Finished!')

    return result

In [70]:
PATH = 'data/IMDB/imdb.pkl'
imdb = pickle.load(open('data/IMDB/imdb.pkl', 'rb'))
G_mdm = nx.convert_matrix.from_numpy_array(imdb['MDM'].astype(int))
G_mam = nx.convert_matrix.from_numpy_array(imdb['MAM'].astype(int))
n_clusters = 3

vanilla_result = vanilla([G_mdm, G_mam], 'imdb', 3)
vanilla_kmeans = [KMeans(n_clusters=n_clusters, random_state=0, n_init=200).fit(vanilla_result[0]),
                 KMeans(n_clusters=n_clusters, random_state=0, n_init=200).fit(vanilla_result[1])]
vanilla_labels = [vanilla_kmeans[0].labels_, vanilla_kmeans[1].labels_]

## vanilla
label = []
for li in imdb['label'].astype(int):
    if li[0] == 1:
        label.append(0)
    elif li[1] == 1:
        label.append(1)
    else:
        label.append(2)

In [78]:
alpha_l = [0.1, 0.1]
alpha_s = [0.1, 0.1]
alpha_u = [0.1, 0.1]
lambda_s = 10
lambda_u = 1e+3
result = dict()
result['imdb'] = debias_mining_model([G_mdm, G_mam], name='imdb', alpha_l=alpha_l, alpha_s=alpha_s, alpha_u=alpha_u, 
                                     lambda_s=1000, lambda_u = 1e+6, ncluster=3, metric='jaccard')
with open('result/sc/model/jaccard_imdb.pickle', 'wb') as f:
    pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)

f_labels = result['imdb'][11]['labels']

start iter
#### Iteration: 0 ; current alpha:  [0.1 0.1] [0.1 0.1] [0.1 0.1]
Iteration: 0  k: 1 Improved minimax risk (arg/max):  [0 1] -0.0
all_masks [1. 1.] 2.0 2.0 2.0
Iteration: 0 ; step reduced minimax?: 0  risk (arg/ max) [0 1] [-0.  0.] ; (arg/ max best):  [0 1] -0.0
alpha_l:  [0.1 0.1] alpha_s:  [0.1 0.1] ; new delta:  [0.5 0.5] [0.5 0.5]  mu:  0.5 ; K:  1
risks:  [-0.  0.]  ; best risk:  [-0.  0.]

NMI scores:  0.0015160559246752108 acc:  0.38
Iteration:  2
#### Iteration: 1 ; current alpha:  [0.3 0.3] [0.3 0.3] [0.3 0.3]
Iteration:  1  k: 2 No minimax risk improvement, current best (arg/max):  [0 1] -0.0
all_masks [1. 1.] 2.0 2.0 2.0
Iteration: 1 ; step reduced minimax?: 1  risk (arg/ max) [0 1] [-0.  0.] ; (arg/ max best):  [0 1] -0.0
alpha_l:  [0.3 0.3] alpha_s:  [0.3 0.3] ; new delta:  [0.5 0.5] [0.5 0.5]  mu:  0.5 ; K:  2
risks:  [-0.  0.]  ; best risk:  [-0.  0.]

NMI scores:  0.0008286989659244619 acc:  0.37690140845070425
Iteration:  3
#### Iteration: 2 ; current alpha

## evaluation for imdb

### result 0.1

vanilla acc: 0.3284507042253521

fair acc: 0.37661971830985913

In [79]:
def lp_diff(vanilla_result, fair_result, ord=None):
    diffs = {}
    if ord == 'fro':
        diffs = 0
        for i in range(vanilla_result.shape[1]):
            residual = min(
                np.linalg.norm(vanilla_result[:, i] + fair_result[:, i], ord=2),
                np.linalg.norm(vanilla_result[:, i] - fair_result[:, i], ord=2)
            )
            diffs += (residual ** 2)
        return np.sqrt(diffs)
    else:
        diff = vanilla_result - fair_result
        return np.linalg.norm(diff, ord=ord)
    
def calc_bias(graphs, vanilla_result, fair_result, sim, metric='jaccard'):
 
    vanilla_result = np.array(vanilla_result)
    fair_result = np.array(fair_result)
    # calculate bias
    vanilla_bias = utils.trace(vanilla_result[1].T @ sim @ vanilla_result[1]) # vanilla bias
    fair_bias = utils.trace(fair_result.T @ sim @ fair_result)  # fair bias
    reduction = 1 - (fair_bias / vanilla_bias)
    return reduction

fair_result = result['imdb'][11]['U_mod']
sim = result['imdb'][11]['L_s']
bias = calc_bias([G_mdm, G_mam], vanilla_result, fair_result, sim)
#diff = [lp_diff(vanilla_result[k], fair_result[k], ord='fro') / np.linalg.norm(vanilla_result[k], ord='fro') for k in range(2)]

### accuracy
def acc(true, pred):
    acc = 0
    for i in range(len(true)):
        if true[i] == pred[i]:
            acc += 1
    return acc / len(true)

vanilla_acc = acc(label, vanilla_labels[1])
vanilla_f1 = f1_score(label, vanilla_labels[1], average='macro')
vanilla_nmi = normalized_mutual_info_score(label, vanilla_labels[1])
print('vanilla acc: {}, nmi: {}, f1: {}'.format(vanilla_acc, vanilla_nmi, vanilla_f1))


## fairness
fair_acc = acc(label, f_labels)
fair_f1 = f1_score(label, f_labels, average='macro')
fair_nmi = normalized_mutual_info_score(label, f_labels)
print('fair acc: {}, nmi: {}, f1: {}, reduction:{}'.format(fair_acc, fair_nmi, fair_f1, bias))



vanilla acc: 0.37633802816901407, nmi: 0.005000645791578157, f1: 0.21611338884827044
fair acc: 0.37915492957746477, nmi: 0.0031802332483697854, f1: 0.19286743876870074, reduction:0.9999764280125271


In [80]:
L = result['imdb'][11]['L']
L_s = result['imdb'][11]['L_s']
U = result['imdb'][11]['U']
U_mod = result['imdb'][11]['U_mod']
KU = result['imdb'][11]['KU']

In [81]:
loss = utils.trace(U[0].T @ L[0] @ U[0]) + utils.trace(U[1].T @ L[1] @ U[1])
loss_sim = utils.trace(U_mod.T @ L_s @ U_mod)
loss_ku = utils.trace(U[0].T @ KU[0] @ U[0]) + utils.trace(U[1].T @ KU[1] @ U[1])

In [82]:
print(loss, loss_sim, loss_ku)

-1.7193893006709167 0.0007369281115751006 1.951075483938202e-14


## DBLP

In [2]:
# read data
PATH = 'data/DBLP/dblp.pkl'
dblp = pickle.load(open(PATH, 'rb'))

G_pap = nx.convert_matrix.from_numpy_array(dblp['PAP'].astype(int))
G_ppp = nx.convert_matrix.from_numpy_array(dblp['PPrefP'].astype(int))
G_ptp = nx.convert_matrix.from_numpy_array(dblp['PTP'].astype(int))
#G_patap = nx.convert_matrix.from_numpy_array(dblp['PATAP'].astype(int))

n_clusters = 4

In [ ]:
vanilla_result = vanilla([G_pap, G_ppp, G_ptp], 'dblp', 4)

In [ ]:

vanilla_result = vanilla([G_pap, G_ppp, G_ptp], 'dblp', 4)
vanilla_kmeans = [KMeans(n_clusters=n_clusters, random_state=0, n_init=200).fit(vanilla_result[i]) 
                  for i in rane(len(vanilla_result))]
vanilla_labels = [vanilla_kmeans[i].labels_ for i in rane(len(vanilla_result))]

## vanilla
label = []
for li in dblp['label'].astype(int):
    if li[0] == 1:
        label.append(0)
    elif li[1] == 1:
        label.append(1)
    else:
        label.append(2)

In [ ]:
alpha_l = [0.1, 0.1]
alpha_s = [0.1, 0.1]
alpha_u = [0.1, 0.1]
lmbda = 0.5
result = dict()
result['imdb'] = debias_mining_model([G_mdm, G_mam], name='imdb', alpha_l=alpha_l,
                                     alpha_s=alpha_s, alpha_u=alpha_u, lmbda=lmbda, ncluster=3, metric='jaccard')
with open('result/sc/model/jaccard_imdb.pickle', 'wb') as f:
    pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)



In [38]:
fair_result = result['imdb'][11]['U_mod']
f_kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=200).fit(fair_result)
f_labels = result['imdb'][11]['labels']

## evaluation for DBLP

In [39]:
def lp_diff(vanilla_result, fair_result, ord=None):
    diffs = {}
    if ord == 'fro':
        diffs = 0
        for i in range(vanilla_result.shape[1]):
            residual = min(
                np.linalg.norm(vanilla_result[:, i] + fair_result[:, i], ord=2),
                np.linalg.norm(vanilla_result[:, i] - fair_result[:, i], ord=2)
            )
            diffs += (residual ** 2)
        return np.sqrt(diffs)
    else:
        diff = vanilla_result - fair_result
        return np.linalg.norm(diff, ord=ord)
    
def calc_bias(graphs, vanilla_result, fair_result, sim, metric='jaccard'):
    vanilla_result = np.array(vanilla_result)
    fair_result = np.array(fair_result)
    # calculate bias
    vanilla_bias = utils.trace(vanilla_result[0].T @ sim @ vanilla_result[0]) # vanilla bias
    fair_bias = utils.trace(fair_result.T @ sim @ fair_result)  # fair bias
    reduction = 1 - (fair_bias / vanilla_bias)
    return reduction


sim = result['imdb'][11]['L_s']
bias = calc_bias([G_mdm, G_mam], vanilla_result, fair_result, sim)
#diff = [lp_diff(vanilla_result[k], fair_result[k], ord='fro') / np.linalg.norm(vanilla_result[k], ord='fro') for k in range(2)]

### accuracy
def acc(true, pred):
    acc = 0
    for i in range(len(true)):
        if true[i] == pred[0][i]:
            acc += 1
    return acc / len(true)

def accf(true, pred):
    acc = 0
    for i in range(len(true)):
        if true[i] == pred[i]:
            acc += 1
    return acc / len(true)

vanilla_acc = acc(label, vanilla_labels)
vanilla_f1 = f1_score(label, vanilla_labels[0], average='macro')
vanilla_nmi = normalized_mutual_info_score(label, vanilla_labels[0])
print('vanilla acc: {}, nmi: {} '.format(vanilla_acc, vanilla_nmi))


## fairness
fair_acc = accf(label, f_labels)
fair_f1 = f1_score(label, f_labels, average='macro')
fair_nmi = normalized_mutual_info_score(label, f_labels)
print('fair acc: {}, nmi: {}, reduction:{}'.format(fair_acc, fair_nmi, bias))



vanilla acc: 0.33887323943661973, nmi: 4.1485791205673264e-05 
fair acc: 0.37661971830985913, nmi: 0.004848365378896624, reduction:0.9991040634126029


### Twitter social/mention

In [3]:
PATH = 'data/Twitter/higgs-social_network.edgelist'
#G_social = nx.read_edgelist('data/Twitter/higgs-{}_network.edgelist'.format('social'), 
#                            create_using=nx.Graph(), nodetype=int, data=(('weight', float),))
G_retweet = nx.read_edgelist('data/Twitter/higgs-{}_network.edgelist'.format('retweet'), 
                             create_using=nx.Graph(), nodetype=int, data=(('weight', float),))
G_reply = nx.read_edgelist('data/Twitter/higgs-{}_network.edgelist'.format('reply'), 
                             create_using=nx.Graph(), nodetype=int, data=(('weight', float),))
#G_mention = nx.read_edgelist('data/Twitter/higgs-{}_network.edgelist'.format('mention'), 
#                             create_using=nx.Graph(), nodetype=int, data=(('weight', float),))

In [9]:
udict = {}
def vanilla(graph, name):
    print("start lcc...")
    lcc = max(nx.connected_components(graph), key=len)  # take largest connected components
    print("start adj...")
    adj = nx.to_scipy_sparse_matrix(graph, nodelist=lcc, dtype='float', format='csc')
    ## unnormalized laplacian
    lap = laplacian(adj)
    
    lap_minus = (-1) * lap
    #lap *= -1
    print("start eigen...")
    _, u = eigsh(lap_minus, which='LM', k=10, sigma=1.0)
    udict[name] = dict()
    udict[name]['eigenvectors'] = u

    with open('result/sc/vanilla_'+name+'.pickle', 'wb') as f:
        pickle.dump(udict, f, protocol=pickle.HIGHEST_PROTOCOL)
    return u

In [5]:
n_clusters = 10
social_result = vanilla(G_reply, 'reply')
retweet_result = vanilla(G_retweet, 'retweet')

In [14]:
vanilla_kmeans_reply = KMeans(n_clusters=n_clusters, random_state=0, n_init=1).fit(social_result)
vanilla_labels_reply = vanilla_kmeans_reply.labels_

vanilla_kmeans_retweet = KMeans(n_clusters=n_clusters, random_state=0, n_init=1).fit(retweet_result)
vanilla_labels_retweet = vanilla_kmeans_retweet.labels_

### Let's debias the mining model

In [28]:
def vanilla(graph, name, n_clusters):
    udict = {}
    if name == 'graph':
        lcc = max(nx.connected_components(graph), key=len)  # take largest connected components
        adj = nx.to_scipy_sparse_matrix(graph, nodelist=lcc, dtype='float', format='csc')
    elif name == 'synthetic':
        adj_1 = graph['adjs'][0]
        adj_2 = graph['adjs'][1]
    elif name == 'imdb':
        #lcc_1 = max(nx.connected_components(graph[0]), key=len)  # take largest connected components
        #lcc_2 = max(nx.connected_components(graph[1]), key=len)
        adj_1 = nx.to_scipy_sparse_matrix(graph[0], nodelist=set(graph[0].nodes), dtype='float', format='csc')
        adj_2 = nx.to_scipy_sparse_matrix(graph[1], nodelist=set(graph[1].nodes), dtype='float', format='csc')
    elif name == 'twitter':
        #lcc_1 = max(nx.connected_components(graph[0]), key=len)  # take largest connected components
        #lcc_2 = max(nx.connected_components(graph[1]), key=len)
        adj_1 = nx.to_scipy_sparse_matrix(graph[0], nodelist=set(graph[0].nodes), dtype='float', format='csc')
        adj_2 = nx.to_scipy_sparse_matrix(graph[1], nodelist=set(graph[1].nodes), dtype='float', format='csc')
    ## unnormalized laplacian
    lap_1 = laplacian(adj_1)
    lap_2 = laplacian(adj_2)
    
    lap1_minus = (-1) * lap_1
    lap2_minus = (-1) * lap_2
    #lap *= -1
    _, u_1 = eigsh(lap1_minus, which='LM', k=n_clusters, sigma=1.0)
    _, u_2 = eigsh(lap2_minus, which='LM', k=n_clusters, sigma=1.0)
    udict[name] = dict()
    u = [u_1, u_2]
    udict[name]['eigenvectors'] = u

    with open('result/sc/vanilla_'+name+'.pickle', 'wb') as f:
        pickle.dump(udict, f, protocol=pickle.HIGHEST_PROTOCOL)
    return u

## 2n+1 parameters
def spectral_clustering(adjs, sims, alpha_l, alpha_s, alpha_u, lambda_s, lambda_u, ncluster, niteration, v0,
                        max_patience, K_ini, K_min, mu,
                        risk_round_factor, reset_optimizer):
        ## initialize
        i = 0
        i_patience = 0

        alpha_l = np.array(alpha_l)
        alpha_s = np.array(alpha_s)
        alpha_u = np.array(alpha_u)


        L = {}
        U = {}
        
        ## save 
        matrices = {0: {}}
        matrices[0]['L'] = {}
        matrices[0]['U'] = {}
        matrices[0]['S'] = {}
        matrices[0]['risk'] = []
        matrices[0]['alpha_l'] = []
        matrices[0]['alpha_s'] = []
        matrices[0]['alpha_u'] = []
        matrices[0]['risk_best'] = []
        matrices[0]['alpha_s_best'] = []
        matrices[0]['alpha_u_best'] = []
        matrices[0]['params'] = []

        for v in range(len(adjs)):
            # initial U
            matrices[0]['S'][v] = alpha_s[v] * lambda_s * laplacian(sims[v]) 
            L[v] = alpha_l[v] * laplacian(adjs[v]) + alpha_s[v] * lambda_s * laplacian(sims[v])
            L_minus = (-1) * L[v]

            _, U[v] = eigsh(L_minus, which='LM', k=ncluster, sigma=1.0, v0=v0)
        
        K = K_ini
        while( i <= niteration) & (i_patience <= max_patience):
            matrices[i + 1] = {}
            matrices[i + 1]['L'] = {}
            matrices[i + 1]['U'] = {}
            matrices[i + 1]['S'] = {}
            matrices[i + 1]['KU'] = {}
            matrices[i + 1]['params'] = []
            matrices[i + 1]['risk'] = []
            matrices[i + 1]['alpha_l'] = []
            matrices[i + 1]['alpha_s'] = []
            matrices[i + 1]['alpha_u'] = []
            matrices[i + 1]['risk_best'] = []
            matrices[i + 1]['alpha_l_best'] = []
            matrices[i + 1]['alpha_s_best'] = []
            matrices[i + 1]['alpha_u_best'] = []
            matrices[i + 1]['nmi_scores'] = {}
            matrices[i + 1]['labels'] = {}
            
            alpha_l_penalty = (alpha_l).astype('float16')
            alpha_s_penalty = (alpha_s).astype('float16')
            alpha_u_penalty = (alpha_u).astype('float16')
            
            print('#### Iteration:', i, '; current alpha: ', alpha_l_penalty, alpha_s_penalty, alpha_u_penalty)
            loss = []
            
            
            for v in range(len(adjs)):
                u1 = sparse.csr_matrix(U[v])
                loss.append((u1.T @ L[v] @ u1).diagonal().sum())
            base_loss = np.array(loss)

            # h,risk,_ = bs_optimal(mu_i)
            risk = np.round(base_loss + 0, risk_round_factor)
            risk_max = np.max(risk)

            # argmax_risks
            argrisk_max = np.arange(risk.shape[0])
            argrisk_max = argrisk_max[risk == risk_max] #consider argmax exactly
        
            ## initialize
            if i == 0:
                risk_max_best = risk_max + 1
                
            if risk_max_best > risk_max: # risk is improved
                # update best risk
                risk_max_best = risk_max 
                argrisk_max_best = argrisk_max 
                risk_best = risk 
                alpha_l_best = alpha_l
                alpha_s_best = alpha_s
                alpha_u_best = alpha_u

                ## resets
                K = np.minimum(K, K_min)
                i_patience = 0
                type_step = 0 #improvement

                print('Iteration:', i,' k:',K,'Improved minimax risk (arg/max): ', argrisk_max_best, risk_max_best)
                #f.write('Iteration:', i,' k:',K,'Improved minimax risk (arg/max): ', argrisk_max_best, risk_max_best)
            else:
                K += 1
                i_patience += 1
                type_step = 1 # no improvement
                print('Iteration: ', i,' k:',K, 'No minimax risk improvement, current best (arg/max): ', 
                      argrisk_max_best, risk_max_best)
                #f.write('Iteration: ', i,' k:',K, 'No minimax risk improvement, current best (arg/max): ', 
                #      argrisk_max_best, risk_max_best)
            
            ## step update
            mask_alpha_l = np.zeros(alpha_l.shape)
            mask_alpha_s = np.zeros(alpha_s.shape)
            mask_alpha_u = np.zeros(alpha_u.shape)
            
            mask_alpha_l[risk >= risk_max_best] = 1
            mask_alpha_s[risk >= risk_max_best] = 1
            mask_alpha_u[risk >= risk_max_best] = 1
        
            step_alpha_l = mask_alpha_l / np.sum(mask_alpha_l)
            step_alpha_s = mask_alpha_s / np.sum(mask_alpha_s)
            step_alpha_u = mask_alpha_u / np.sum(mask_alpha_u)
            print('all_masks', mask_alpha_l, np.sum(mask_alpha_l), np.sum(mask_alpha_s), np.sum(mask_alpha_u))
                
            ##############    Save lists  ##############
            #weight vector updated after save lists
            matrices[i + 1]['params'].append([type_step, K, mu])
            matrices[i + 1]['risk'].append(risk)
            matrices[i + 1]['alpha_l'].append(alpha_l)
            matrices[i + 1]['alpha_s'].append(alpha_s)
            matrices[i + 1]['alpha_u'].append(alpha_u)
            matrices[i + 1]['risk_best'].append(risk_best)
            matrices[i + 1]['alpha_l_best'].append(alpha_l_best)
            matrices[i + 1]['alpha_s_best'].append(alpha_s_best)
            matrices[i + 1]['alpha_u_best'].append(alpha_u_best)
            
            print('Iteration:', i, '; step reduced minimax?:', type_step, ' risk (arg/ max)', argrisk_max,
                  risk[argrisk_max], '; (arg/ max best): ', argrisk_max_best, risk_max_best)
            print('alpha_l: ', alpha_l, 'alpha_s: ', alpha_s, '; new delta: ', step_alpha_l, step_alpha_s, ' mu: ', mu, '; K: ', K)
            print('risks: ', risk, ' ; best risk: ', risk_best)
            print()

            
            ############################################
            for view, u in U.items():
                view_kmeans = KMeans(n_clusters=u.shape[1], random_state=42, n_init=5).fit(u)
                view_labels = view_kmeans.labels_
                matrices[i + 1]['labels'][view] = view_labels
                #matrices[i + 1]['nmi_scores'][view] = normalized_mutual_info_score(view_labels, vanilla_labels[view])
                #matrices[i + 1]['nmi_scores'][view] = normalized_mutual_info_score(label, view_labels)

            #print("NMI scores: ", matrices[i + 1]['nmi_scores'])
            ### UPDATE WEIGHTING_VECTOR ###
            alpha_l = (1 - mu) * alpha_l + step_alpha_l * mu / K
            alpha_s = (1 - mu) * alpha_s + step_alpha_s * mu / K
            alpha_u = (1 - mu) * alpha_u + step_alpha_u * mu / K
            

            i += 1
                

            for j in range(len(adjs)):
                KU = sum(np.matmul(U[k], U[k].T) for k in range(len(U)) if k != j)
                lap = alpha_l[j] * L[j] - alpha_u[j] * lambda_u * KU + alpha_s[j] * lambda_s * laplacian(sims[j])
                
                lap_minus = (-1) * lap
                _, U[j] = eigsh(lap_minus, which='LM', k=ncluster, sigma=1.0, v0=v0)
                L[j] = np.array(lap)

                matrices[i]['L'][j] = L[j]
                matrices[i]['U'][j] = U[j]
                matrices[i]['KU'][j] = KU
                matrices[i]['S'][j] = alpha_s[j] * lambda_s * laplacian(sims[j]) 

            print('Iteration: ', i+1)
            #f.write('Iteration: ', i+1)



        #return matrices
        return matrices[i]
    
def debias_mining_model(graphs, name, alpha_l, alpha_s, alpha_u, lmbda, ncluster, niter=20, metric=None):
    # build adjacency matrix
    adj_1 = nx.to_scipy_sparse_matrix(graphs[0], nodelist=set(graphs[0].nodes), dtype='float', format='csc')
    adj_2 = nx.to_scipy_sparse_matrix(graphs[1], nodelist=set(graphs[1].nodes), dtype='float', format='csc')

    # build similarity matrix
    sim_1 = utils.get_similarity_matrix(adj_1, metric=metric)
    sim_2 = utils.get_similarity_matrix(adj_2, metric=metric)
    
    adjs = np.array([adj_1, adj_2])
    sims = np.array([sim_1, sim_2])
    # debias spectral clustering
    
    #alpha_l = alpha_
    #alpha_s = [alpha, alpha]
    #alpha_u = [alpha, alpha]
    lambda_s = lmbda
    lambda_u = lmbda
    
    # V, U = sc.debias_alg(adj, sim, alpha, ncluster=10, v0=v0[name])
    # model = DebiasModelFair(adjs, sims, ncluster=10, niteration=niter, v0=v0[name])
    result = spectral_clustering(adjs, sims, alpha_l, alpha_s, alpha_u, lambda_s, lambda_u,
                                 ncluster=n_clusters, niteration=niter, v0=None,
                                 max_patience=15, K_ini=1, K_min=20, mu=0.5,
                                 risk_round_factor=3, reset_optimizer=False)
    

    print('dataset: {}\t metric: {} similarity'.format(name, metric))
    print('Finished!')

    return result

In [5]:
n_clusters = 10
social_result = vanilla(G_reply, 'reply')
retweet_result = vanilla(G_retweet, 'retweet')

In [14]:
vanilla_kmeans_reply = KMeans(n_clusters=n_clusters, random_state=0, n_init=1).fit(social_result)
vanilla_labels_reply = vanilla_kmeans_reply.labels_

vanilla_kmeans_retweet = KMeans(n_clusters=n_clusters, random_state=0, n_init=1).fit(retweet_result)
vanilla_labels_retweet = vanilla_kmeans_retweet.labels_

In [29]:
alpha_l = [1, 0.03]
alpha_s = [1, 0.03]
alpha_u = [1, 0.03]
lmbda = 5
result = dict()
result['imdb'] = debias_mining_model([G_reply, G_retweet], name='twitter', alpha_l=alpha_l,
                                     alpha_s=alpha_s, alpha_u=alpha_u, lmbda=lmbda, ncluster=3, metric='jaccard')
with open('result/sc/model/jaccard_imdb.pickle', 'wb') as f:
    pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)

"""f_kmeans = [KMeans(n_clusters=n_clusters, random_state=0, n_init=200).fit(result['imdb']['U'][0]),
            KMeans(n_clusters=n_clusters, random_state=0, n_init=200).fit(result['imdb']['U'][1])]
f_labels = [f_kmeans[0].labels_, f_kmeans[1].labels_]"""

KeyboardInterrupt: 